In [1]:
import pandas as pd
import numpy as np
from jupyter_dash import JupyterDash
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import folium
from folium.plugins import HeatMap, FastMarkerCluster
import dash_bootstrap_components as dbc

/home/christos/anaconda3/lib/python3.8/site-packages/dash_bootstrap_components/table.py:1: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [2]:
#external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
external_stylesheets = [dbc.themes.CERULEAN]

In [4]:
df= pd.read_csv('../data/Crime_Data_from_2010_to_Present(cleaned).csv')

In [5]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [6]:
df.head()

,Year,Month,Area,Crime Code,Crime,Vict Age,Vict Descent,Weapon Code,Weapon,Premises Code,Premises,LAT,LON
0,2010,1,Hollywood,900,VIOLATION OF COURT ORDER,47,White,102.0,HAND GUN,101.0,STREET,34.1016,-118.3295
1,2010,1,Central,122,"RAPE, ATTEMPTED",47,Hispanic,400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",103.0,ALLEY,34.0387,-118.2488
2,2010,1,Central,624,BATTERY - SIMPLE ASSAULT,38,Black,400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",101.0,STREET,34.0640,-118.2375
3,2010,1,Central,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",24,White,NaN,NaN,102.0,SIDEWALK,34.0409,-118.2609
4,2010,1,Central,755,BOMB SCARE,29,Black,500.0,UNKNOWN WEAPON/OTHER WEAPON,738.0,LIBRARY,34.0502,-118.2540


In [7]:
crimes = df['Crime'].value_counts().head(20).reset_index()

### Creating the Dashboard

In [8]:
def map_from_df(data):
    base_map = folium.Map(location=[34.0522, -118.2437], zoom_start=8,
                          tiles='Stamen Terrain')
   
    
    heat_data = [[row['LAT'], row['LON']] for index, row in data.iterrows()]
    FastMarkerCluster(heat_data).add_to(base_map)
    return base_map 

def value_filter3(df, crime,v_min,v_max):
    df = df[(df['Crime'].isin(crime))]
    return df[((df.Year >= v_min) & (df.Year <=v_max))]

def value_filter1(df,v_min,v_max):
    return df[((df.Year >= v_min) & (df.Year <=v_max))]

def value_filter2(df, v_min,v_max):
    df = df.groupby(['Crime','Year'], as_index=False)['Vict Age'].count().sort_values(by='Vict Age')
    return df[((df.Year >= v_min) & (df.Year <=v_max))]

def value_pie(df, dropvalue,v_min,v_max):
    df['age bins'] = pd.cut(df['Vict Age'], bins=3, labels = ['Young','Middle Age','Old'])
    if dropvalue == 'Weapon':
        df.dropna(inplace=True)
    return df[((df.Year >= v_min) & (df.Year <=v_max))]

In [9]:
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

app.layout = dbc.Container([
 
#FIRST ROW----------------------(HEADER)-------------------------------------------------------  
    dbc.Row([ 
       dbc.Col(html.H1('Crimes Against Women Population in Los Angeles', 
                        className = 'text-center text-primary mb-4'),
                width=12)
    ]),
                    
#SECOND ROW------------------(LABEL + DROPDOWN + PIE CHART)------------------------------------ 
    dbc.Row([      
        dbc.Col([html.Label(['Select Element to Inspect:'], style={'font-weight': 'bold',"textDecoration": "underline", 
                                       'text-align':'center'}),
            dcc.Dropdown(
                id='dropdown',
                options=[
                    {'label': 'Crime Commited', 'value': 'Crime'},
                    {'label': 'Weapon Used Against Victims', 'value': 'Weapon'},
                    {'label': 'Premises of Crime', 'value': 'Premises'},
                    {'label': 'Crimes by Year', 'value': 'Year'},
                    {'label': 'Victim Age', 'value': 'age bins'},
                    {'label': 'Victim Descent', 'value':'Vict Descent'},
        
        ],
        value='age bins',
        style={'width':"50%"}
    ),
               
        html.Br(),
        html.H4('Overview',className = 'text-center text-primary mb-4'),
        
        dcc.Graph(id="pie", figure={}),                              #pie plot
        html.Br(),
        
                ]),
        ]),

#THIRD ROW-------------------(LABEL+RANGESLIDER)-------------------------------------------        
        dbc.Row([           
            dbc.Col([html.Label('Select Year:', style={'font-weight': 'bold',"textDecoration": "underline"}, 
                                className ='text-center'),
                dcc.RangeSlider(
                    id='range-slider',
                        min=2010,
                        max=2021,
                        step=1,
                        value=[2010, 2011],
                    marks={2010:{"label":'2010'},
                           2011:{"label":'2011'},
                           2012:{"label":'2012'},
                           2013:{"label":'2013'},
                           2014:{"label":'2014'},
                           2015:{"label":'2015'},
                           2016:{"label":'2016'},
                           2017:{"label":'2017'},
                           2018:{"label":'2018'},
                           2019:{"label":'2019'},
                           2020:{"label":'2020'},
                           2021:{"label":'2021'}},className = 'mb-4'
                        
            ), 
        ], width={'size':6, 'offset':3}),
    ]),
#FOURTH ROW--------------------(2 BARGRAPHS)----------------------------------------------    
    dbc.Row([

     
    #first column ---------------------------------------------------       
        dbc.Col([dcc.Graph(id="barplot", figure={}),               #first barplot, crimes
             ]),
    
            html.Br(),
    
    #second column--------------------------------------------------       
            dbc.Col([dcc.Graph(id="barplot2", figure={}) ,              #second barplot, descent
            html.Br()
                    ])
        ]),

#FIFTH ROW--------------------(CHECKLIST)-----------------------------------------------   
    dbc.Row([
        dbc.Col([html.Label("Select Crime to Inspect:",
                   style={'font-weight': 'bold',"textDecoration": "underline"}),
            
            dcc.Checklist(id='my-checklist',value=['BATTERY - SIMPLE ASSAULT'], 
                              options = [{'label':x, 'value':x}
                                        for x in crimes['index']],
                          labelClassName="mr-3")
                ],width={'size':8, 'offset':2}, className='mb-4')
    ]),

#SIXTH ROW---------------------(HEADER)----------------------------------------------------        
    dbc.Row([
            dbc.Col([html.H3('Areas of Recorded Crimes',
                             className = 'text-center text-primary mb-4')], width=12)
            ]),
         
    
#SEVENTH ROW----------------------------(HEATMAP)------------------------------------------    
    dbc.Row([
        dbc.Col(html.Iframe(id='map', width='100%', height=600),  width={"size":8,'offset':2})     #heatmap
     ])
  
        ], fluid=True)




#callback for pie----------------------------------------------------------------------

@app.callback(
    Output("pie", component_property ='figure'),
     [Input('dropdown', component_property ='value'),
     Input('range-slider', component_property ='value')]
)
def update_graph(value,value1):
    dff = df
    v_min = value1[0]
    v_max = value1[1]
    figpie = px.pie(value_pie(dff,value,v_min,v_max),  names= value, 
                       hole=.1)
    figpie.update_traces(textposition='inside', textinfo='percent', hoverinfo='percent+label')
    return figpie

#callback for histplots---------------------------------------------------------

@app.callback(
    [Output("barplot", component_property ='figure'),      #output barplot with crimes
     Output("barplot2", component_property ='figure')],
     Input('range-slider', component_property ='value')
)
def update_graph(value):
    dff = df
    v_min = value[0]
    v_max = value[1]
    fig = px.histogram((value_filter2(dff, v_min,v_max)).tail(30), x='Vict Age', y= 'Crime', 
                        height=500, 
                   labels={'Vict Age':'Victims','Crime': ''},title ='Top Crimes per Year')
    
    fig2 = px.histogram(value_filter1(dff,v_min,v_max),  y= 'Vict Descent', 
                       title = 'Number of Victims per Ethnicity', height=500, 
                   labels={'count':'Victims'}, color= 'Vict Descent')
    return fig, fig2

#callback for map------------------------------------------------------------

@app.callback(
     Output('map', component_property ='srcDoc'),
     [Input('my-checklist', component_property ='value'),
     Input('range-slider', component_property ='value')]
)
def update_graph(crime,value):
    dff=df
    v_min=value[0]
    v_max=value[1]
    
    df_cut = value_filter3(dff, crime,v_min,v_max)
    base_map = map_from_df(df_cut)
    return base_map._repr_html_()

    
    
app.run_server(mode ='external',host="localhost", port=8051)

Dash app running on http://localhost:8051/
